In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np 
import pandas as pd 


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Machine Learning from Titanic Disaster

# First and foremost, we want to understand what data we have:

### PassengerId - Identifier 


### Survived: This is our target variable, indicating whether a passenger survived (1) or didn't survive (0).

### Pclass: This represents the socio-economic class of the passenger, with three categories: 1st class (Upper), 2nd class (Middle), and 3rd class (Lower).

### Name: The passenger's name is included in this column. While names themselves may not directly correlate with survival, you can extract titles (e.g., Mr., Mrs.) which might provide additional information.

### Sex: This column indicates the gender of the passenger, with values "male" and "female."

### Age: Age of the passenger, which is fractional if less than 1 and in the form of xx.5 if estimated.

### SibSp: This column represents the number of siblings or spouses traveling with each passenger. It defines family relations based on siblings and spouses.

### Parch: Indicates the number of parents or children traveling with each passenger. It defines family relations based on parents and children.

### Ticket: This column contains the ticket number of the passenger, which may or may not provide predictive information.

### Fare: The fare paid by each passenger for the ticket.

### Cabin: Cabin number, which might provide information about the passenger's location on the ship. However, it has many missing values.

### Embarked: This column represents the port of embarkation for each passenger and has three possible values: C (Cherbourg), Q (Queenstown), and S (Southampton).


In [ ]:
# Let's load our dataset into a DataFrame
raw_data = pd.read_csv('/kaggle/input/titanic/train.csv')
raw_data 

In [ ]:
# Lets explore the raw data a little
raw_data.info()

In [ ]:
# Lets get some quick summary stats for some our continuous numberical data
raw_data[['Age','Fare']].describe()



In [ ]:
sns.displot(raw_data, x='Age',hue='Survived', kde=True)

In [ ]:
sns.displot(raw_data, x='Fare',hue="Survived")
# looks like the more you paid, the higher the liklihood it was that you survived


In [ ]:
sns.displot(raw_data, x='Pclass',hue='Survived')
# looks like the higher the class, the higher the liklihood it was that you survived

In [ ]:
# lets assess the distributions of our features
plt.figure(figsize=(20,15))
sns.pairplot(raw_data)

In [ ]:
#plt.figure(figsize=(20,15))
#sns.heatmap(raw_data.corr(),annot=True)